In [2]:
import requests
import hashlib
import time
import json

In [42]:
API_KEY = "4YCF385ZNXLEHADRUSFV"
API_SECRET = "YcjAfd9x7S$$qX^4u#n$rvLD$X6c^pT87ShWvF3V"

In [48]:
# Generate the timestamp
timestamp = str(int(time.time()))
auth_hash = hashlib.sha1((API_KEY + API_SECRET + timestamp).encode('utf-8')).hexdigest()

headers = {
    "X-Auth-Date": timestamp,
    "X-Auth-Key": API_KEY,
    "Authorization": auth_hash,
    "User-Agent": "PodcastIndex-Client"
}

# Step 1: Get Trending Podcasts
url = "https://api.podcastindex.org/api/1.0/podcasts/trending"
params = {"max": 200}  # Get up to 200 trending podcasts

response = requests.get(url, headers=headers, params=params)
trending_podcasts = response.json().get("feeds", [])

print(f"Retrieved {len(trending_podcasts)} trending podcasts.")

Retrieved 200 trending podcasts.


In [49]:

# Step 1: Download trending podcasts
url = "https://api.podcastindex.org/api/1.0/podcasts/trending"
response = requests.get(url, headers=headers, params=params)
trending_data = response.json()

# Step 2: Extract relevant fields
trending_data = response.json()
trending_podcasts = trending_data.get("feeds", [])

simplified_podcasts = [
    {
        "title": feed.get("title"),
        "url": feed.get("url"),
        "description": feed.get("description"),
        "id": feed.get("id"),
        "author": feed.get("author"),
    }
    for feed in trending_podcasts
]


# Step 3: Save to JSON file
output_path = "top_podcasts.json"
with open(output_path, "w") as f:
    json.dump(simplified_podcasts, f, indent=4)

print(f"Saved {len(simplified_podcasts)} podcasts to {output_path}")




Saved 200 podcasts to top_podcasts.json


In [50]:
with open("top_podcasts.json", "r") as f:
    podcasts = json.load(f)

all_episodes = []

for podcast in podcasts:
    feed_id = podcast.get("id")
    podcast_title = podcast.get("title")

    print(f"Fetching episodes for: {podcast_title}")

    response = requests.get(
        "https://api.podcastindex.org/api/1.0/episodes/byfeedid",
        headers=headers,
        params= {"id": feed_id, "max": 1} #more episodes from one podcast - change max
    )

    if response.status_code == 200:
        episodes = response.json().get("items", [])
        for ep in episodes:
            all_episodes.append({
                "podcast": podcast_title,
                "episode_title": ep.get("title"),
                "audio_url": ep.get("enclosureUrl"),
                "id": ep.get("id")
            })
    

# Save to top_episodes.json
with open("top_episodes.json", "w") as f:
    json.dump(all_episodes, f, indent=4)

print(f"✅ Saved {len(all_episodes)} episodes to top_episodes.json")


Fetching episodes for: Calming Anxiety
Fetching episodes for: Spirit Force
Fetching episodes for: Ethan Brings You the World
Fetching episodes for: Promo Estereo Podcast
Fetching episodes for: Craave Raasi Palan
Fetching episodes for: A hombros de gigantes
Fetching episodes for: 高詩佳故事學堂│經典 ‧ 智慧 ‧ 閱讀素養
Fetching episodes for: このは屋
Fetching episodes for: このは屋
Fetching episodes for: High Performance Mindset | Learn from World-Class Leaders, Consultants, Athletes & Coaches about Mindset
Fetching episodes for: PHNX Arizona Cardinals Podcast
Fetching episodes for: Eight Years in Paris
Fetching episodes for: The Politics Prepping and Paranormal Podcast
Fetching episodes for: It's a Hard Knicks Life
Fetching episodes for: The John Batchelor Show
Fetching episodes for: Scott Ritter Extra: Ask the Inspector
Fetching episodes for: WRFH/Radio Free Hillsdale 101.7 FM
Fetching episodes for: The Young Turks
Fetching episodes for: The Vibe You're In
Fetching episodes for: Real America’s Voice
Fetching 

In [ ]:
import subprocess
import os
output_dir = "transcripts"
os.makedirs(output_dir, exist_ok=True)

# Load episodes file
with open("top_episodes.json", "r") as f:
    episodes = json.load(f)

# Iterate through each podcast entry
for episode in episodes:
    podcast_name = episode.get("podcast", "unknown_podcast")
    episode_title = episode.get("episode_title", "unknown_episode")
    audio_url = episode.get("audio_url")

    # Construct output filename 
    safe_title = episode_title.replace(" ", "_").replace("/", "_")
    safe_podcast = podcast_name.replace(" ", "_").replace("/", "_")
    output_file = os.path.join(output_dir, f"{safe_podcast}_{safe_title}.json")

    # Check if the audio_url is valid
    if audio_url and audio_url.strip():
        print(f"Transcribing {episode_title} from {audio_url}")
        try:
            # Run podscript command
            result = subprocess.run(
                [
                    "/Users/terezasaskova/go/bin/podscript",
                     "assembly-ai",
                     "--from-url", audio_url,
                    "--output", output_file
                ],
                check=True,
                capture_output=True,
                text=True
            )

            print(f"Transcript saved to {output_file}")

        except subprocess.CalledProcessError as e:
            print(f"Subprocess error:\n{e.stderr}")
        except Exception as e:
            print(f"Exception occurred: {e}")
    else:
        print(f"No valid audio URL found for {episode_title}")

print("Transcription process complete.")


Transcribing I nurture balance in all I think, feel and do from https://dts.podtrac.com/redirect.mp3/api.spreaker.com/download/episode/66343448/i_nurture_balance_in_all_i_think_feel_and_do.mp3
Transcript saved to transcripts/Calming_Anxiety_I_nurture_balance_in_all_I_think,_feel_and_do.json
Transcribing Adventures of Captain Epoch Ep 2-6: The Fed and the Titanic from https://dts.podtrac.com/redirect.mp3/api.spreaker.com/download/episode/66345491/adventures_of_captain_epoch_ep_2_6_thefedandthetitanic.mp3
Transcript saved to transcripts/Spirit_Force_Adventures_of_Captain_Epoch_Ep_2-6:_The_Fed_and_the_Titanic.json
Transcribing 05-30-25 Part 12 from https://dts.podtrac.com/redirect.mp3/api.spreaker.com/download/episode/66344036/untitled_session_105_mixdown_12.mp3
Transcript saved to transcripts/Ethan_Brings_You_the_World_05-30-25_Part_12.json
Transcribing AÑORANZAS MUSICALES || PROGRAMA DE ESTRENO from https://dts.podtrac.com/redirect.mp3/api.spreaker.com/download/episode/66345135/a_oranza

In [16]:
import whisper
import os
import subprocess
from pathlib import Path
import sys
import tempfile


# Load Whisper model once
model = whisper.load_model("base")  # or "tiny", "small"

# Paths
AUDIO_DIR = Path("audio")
TRANSCRIPT_DIR = Path("transcripts")
os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs(TRANSCRIPT_DIR, exist_ok=True)

def download_audio(url, output_path):
    subprocess.run([
        "yt-dlp",
        "-x", "--audio-format", "mp3",
        "-o", str(output_path),
        url
    ], check=True)

def transcribe(audio_path):
    result = model.transcribe(str(audio_path))
    return result["text"]

In [18]:
import json

# Load JSON file
with open("top_podcasts.json", "r") as f:
    data = json.load(f)

for episode in data:
    ep_id = episode["id"]
    title = episode["title"]
    url = episode["url"]

    print(f"🔄 Processing: {title} ({url})")

    # Clean filename
    filename_base = f"{ep_id}_{title.replace(' ', '_')}"
    audio_path = Path(AUDIO_DIR) / f"{filename_base}.mp3"
    transcript_path = Path(TRANSCRIPT_DIR) / f"{filename_base}.json"

    # Skip if already done
    if transcript_path.exists():
        print("✅ Already transcribed, skipping.")
        continue

    try:
        with tempfile.TemporaryDirectory() as tmpdir:
            # Build a unique output pattern inside the temp dir
            output_path = os.path.join(tmpdir, "%(title)s.%(ext)s")

            subprocess.run([
                "yt-dlp",
                "-x", "--audio-format", "mp3",
                "-o", output_path,
                url
            ], check=True)

            # Find the downloaded file
            audio_files = [f for f in os.listdir(tmpdir) if f.endswith(".mp3")]
            if not audio_files:
                raise FileNotFoundError("❌ No MP3 downloaded.")

            temp_audio_path = os.path.join(tmpdir, audio_files[0])

            print("🧠 Transcribing...")
            result = model.transcribe(temp_audio_path)

            with open(transcript_path, "w") as out:
                json.dump(result, out)

            print(f"✅ Saved transcript: {transcript_path}")

    except subprocess.CalledProcessError:
        print("❌ Failed to download audio.")
        continue
    except Exception as e:
        print(f"❌ Transcription failed: {e}")

🔄 Processing: Calming Anxiety (https://www.spreaker.com/show/4110266/episodes/feed)
[SpreakerShow] Extracting URL: https://www.spreaker.com/show/4110266/episodes/feed
[download] Downloading playlist: 4110266
[SpreakerShow] 4110266: Downloading JSON page 1
[SpreakerShow] 4110266: Downloading JSON page 2
[SpreakerShow] 4110266: Downloading JSON page 3
[SpreakerShow] 4110266: Downloading JSON page 4
[SpreakerShow] 4110266: Downloading JSON page 5
[SpreakerShow] 4110266: Downloading JSON page 6
[SpreakerShow] 4110266: Downloading JSON page 7
[SpreakerShow] 4110266: Downloading JSON page 8
[SpreakerShow] 4110266: Downloading JSON page 9
[SpreakerShow] 4110266: Downloading JSON page 10
[SpreakerShow] 4110266: Downloading JSON page 11
[SpreakerShow] 4110266: Downloading JSON page 12
[SpreakerShow] 4110266: Downloading JSON page 13
[SpreakerShow] 4110266: Downloading JSON page 14
[SpreakerShow] 4110266: Downloading JSON page 15
[SpreakerShow] 4110266: Downloading JSON page 16
[SpreakerShow] 41



ERROR: unable to write data: [Errno 28] No space left on device



[download] Downloading item 108 of 1934
[info] 64343017: Downloading 1 format(s): mp3
[download] Destination: /var/folders/73/tnmscj2j2k1d5vykfvr7gp7r0000gn/T/tmpvswe83dk/I can handle what ever comes my way with calmness and understanding.mp3
[download]   8.4% of   23.69MiB at    5.31MiB/s ETA 00:04  



ERROR: unable to write data: [Errno 28] No space left on device



[download] Downloading item 109 of 1934
[info] 64342114: Downloading 1 format(s): mp3
[download] Destination: /var/folders/73/tnmscj2j2k1d5vykfvr7gp7r0000gn/T/tmpvswe83dk/I am strong and resilient yet kind and understanding.mp3
[download]   2.0% of   24.49MiB at    4.76MiB/s ETA 00:05  
[download] Downloading item 110 of 1934
[info] 64323339: Downloading 1 format(s): mp3


...


[download] Downloading item 26 of 100
[generic] Extracting URL: https://traffic.omny.fm/d/clips/566281f8-200e-4c9f-8378-a4870055423b/f9f072c2-79c5-467c-a834-a702...5-b2cf00802c07%22%7D
[generic] cc330d8c-34f9-4c06-adf5-b2cf00802c07: Downloading webpage
[redirect] Following redirect to https://ondemand2-us-dal03.cdnstream1.com/d/clips/566281f8-200e-4c9f-8378-a4870055423b/f9f072c2-79c5-467c-a834-a7020168a89d/cc330d8c-34f9-4c06-adf5-b2cf00802c07/audio/direct/t1746000178/BONUS_PODCAST_Our_Extended_Chat_With_Jamila_Rizvi_On_Living_with_a_Broken_Brain.mp3?t=1746000178&publisher=sca&awEpisodeId=cc330d8c-34f9-4c06-adf5-b2cf00802c07&awEpisodeTitle=BONUS%20PODCAST%3A%20Our%20Extended%20Chat%20With%20Jamila%20Rizvi%20On%20Living%20with%20a%20Broken%20Brain&awCollectionId=f9f072c2-79c5-467c-a834-a7020168a89d&in_playlist=f6d85f68-0990-40bb-aeee-a70201691cca&utm_source=Podcast&listeningSessionID=0CD_382_151__e6bfd1125c9cbdb27a83958e210e8ea28437b148
[generic] Extracting URL: https://ondemand2-us-dal

ERROR: unable to open for writing: [Errno 28] No space left on device: '/var/folders/73/tnmscj2j2k1d5vykfvr7gp7r0000gn/T/tmp8gjdkvxz/BONUS PODCAST： Our Extended Chat With Jamila Rizvi On Living with a Broken Brain.mp3.part'



[download] Downloading item 27 of 100
[generic] Extracting URL: https://traffic.omny.fm/d/clips/566281f8-200e-4c9f-8378-a4870055423b/f9f072c2-79c5-467c-a834-a702...4-b2ce007a830d%22%7D
[generic] 95978c77-9f35-4584-9a24-b2ce007a830d: Downloading webpage
[redirect] Following redirect to https://ondemand2-us-dal03.cdnstream1.com/d/clips/566281f8-200e-4c9f-8378-a4870055423b/f9f072c2-79c5-467c-a834-a7020168a89d/95978c77-9f35-4584-9a24-b2ce007a830d/audio/direct/t1745913925/Why_Does_Your_Breath_Smell_Like_Poo.mp3?t=1745913925&publisher=sca&awEpisodeId=95978c77-9f35-4584-9a24-b2ce007a830d&awEpisodeTitle=%E2%80%9CWhy%20Does%20Your%20Breath%20Smell%20Like%20Poo%3F%E2%80%9D&awCollectionId=f9f072c2-79c5-467c-a834-a7020168a89d&in_playlist=f6d85f68-0990-40bb-aeee-a70201691cca&utm_source=Podcast&listeningSessionID=0CD_382_151__7994c8582200c8733980815c2bae5771bfe0f86c
[generic] Extracting URL: https://ondemand2-us-dal03.cdnstream1.com/d/clips/566281f8-200e-4c9f-8378-a4870055423b/f9f072c2-7...4-b2ce00


ERROR: Interrupted by user


KeyboardInterrupt: 